# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
Our task is to classify the students into 2 classes of passing or not-passing, this makes the problem a binary classification task.
If we were to additionally predict when a student would require assistance, or to what degree someone needs assistance as a quantitative measure, this could become a regression task

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] =='yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] !='yes'].shape[0]

# TODO: Calculate graduation rate
grad_rate = n_passed * 100.0 / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


Given that the classes are imbalanced, 67.09% of the students are passing, we can use the F1 score of predicting only yes for all values as a benchmark for our models

In [3]:
from sklearn.metrics import f1_score
print "\nF1 score for predicting all 'yes': {:.4f}".format(
    f1_score(y_true = ['yes']*n_passed + ['no']*n_failed, y_pred = ['yes']*n_students, pos_label='yes', average='binary'))


F1 score for predicting all 'yes': 0.8030


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, stratify=y_all, test_size=num_test, random_state=42)
# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print "Grad rate of train set: {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Grad rate of test set: {:.2f}%".format(100 * (y_test == 'yes').mean())

Training set has 300 samples.
Testing set has 95 samples.
Grad rate of train set: 67.00%
Grad rate of test set: 67.37%


Since the classes are imbalanced, we additionally use the stratify parameter to maintain that imbalance across the test/training split.

## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

I chose to use the following algorithms:
  1. ** Support Vector Machine Classifier **:
      * *General Applications* : 
      
          SVMs have been used successfuly for applications of text categorization, classification of proteins for biological sciences as well as image segmentation tasks.
          
      * *Strengths* :
      
          1. decision function is created using a subset of the training points, thus memory efficient
          2. effective in high dimensional spaces, in our case we have 48 dimensions and only 300 data points
          3. kernel functions allow modelling of non linear relations
          4. since algorithm maximizes margin of decision boundary, the models are more robust
      
      * *Weaknesses* :
      
          1. difficult to interpret relation of the features to the model when the kernel functions are non-linear
          2. They do not provide probibilty estimates directly, need to do cross-validation to find them
      
      * *Reasons for choosing* :
      
          I chose to use an SVM classifier as it creates robust models in high dimensional spaces.
  2. ** K Nearest Neighbors Classifier **:
      * *General Applications* : 
      
          KNN algorithms have been useful for applications which involve finding similar objects, they have successful applications on building document search retrival algorithms as well as recommendation systems. 
      * *Strengths* :
      
          1. simple algorithm to understand
          2. classifier can be updated online cheaply when new data points become available.
          3. classes do not have to be linearly seperable since the classifier is not creating a generalized model.
      * *Weaknesses* :
      
          1. may not perform well in high dimensional spaces if the number of training points are low
          2. prediction cost is expensive since we need to compute and find the k nearest neighbors of a data point when querying, indexing of training points is required.
      * *Reasons for choosing* :
      
          I chose to use KNN because its known to perform fairly well and further more, information about the neareast neighbors of the predicted data point can be used to interpret and justify the classification more intuitively to the school board. 
  3. ** Ensemble Methods (Bagging) ** : 
      * *General Applications* : 
      
          Ensemble methods have been used regularly in machine learning competitions such as the netflix movie recommendations competition, KDD 2009 collaborative filter and numerous kaggle competitions.
          
      * *Strengths* :
      
          1. reduces the variance of the base estimator.
          2. reduces overfitting to the dataset by using multiple classifiers on random subsets
          3. provides a simple way to improving the performance of the base estimator without doing grid search on it
          
      * *Weaknesses* :
      
          1. may perform worse than the underlying classifier in some cases.
          2. only useful when the individual classifiers are not biased towards a particular class.
      * *Reasons for choosing* :
          
          i wanted to use the ensemble method with svms since im using linear svms as the base model i expect the ensemble method to capture some of the non linearity from the dataset.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)
    return (end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return (f1_score(target.values, y_pred, pos_label='yes'), end - start)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_time = train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    train_f1, train_pred_time = predict_labels(clf, X_train, y_train)
    test_f1, test_pred_time = predict_labels(clf, X_test, y_test)
    print "F1 score for training set: {:.4f}.".format(train_f1)
    print "F1 score for test set: {:.4f}.".format(test_f1)
    return (len(X_train), train_time, test_pred_time,  train_f1, test_f1)

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [16]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
# TODO: Initialize the three models
clf_A = svm.SVC(random_state=42)
clf_B = neighbors.KNeighborsClassifier()
clf_C = ensemble.BaggingClassifier(svm.SVC(), random_state=42)
# TODO: Set up the training set sizes
training_sizes = [100,200, 300]
classifiers = [clf_A, clf_B, clf_C]
results_table = {}
for clf in classifiers:
    results_table[clf.__class__.__name__] = []
for size in training_sizes:
    X_train_subset, y_train_subset = X_train[:size], y_train[:size]
    for clf in classifiers:
        results = train_predict(clf, X_train_subset, y_train_subset, X_test, y_test )
        results_table[clf.__class__.__name__].append(results)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0075 seconds
Made predictions in 0.0013 seconds.
Made predictions in 0.0013 seconds.
F1 score for training set: 0.8354.
F1 score for test set: 0.8025.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0008 seconds
Made predictions in 0.0018 seconds.
Made predictions in 0.0017 seconds.
F1 score for training set: 0.8252.
F1 score for test set: 0.7586.
Training a BaggingClassifier using a training set size of 100. . .
Trained model in 0.0635 seconds
Made predictions in 0.0282 seconds.
Made predictions in 0.0118 seconds.
F1 score for training set: 0.8667.
F1 score for test set: 0.7919.
Training a SVC using a training set size of 200. . .
Trained model in 0.0074 seconds
Made predictions in 0.0035 seconds.
Made predictions in 0.0029 seconds.
F1 score for training set: 0.8431.
F1 score for test set: 0.8105.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

In [ ]:
# Generate Markdown tables from the results
for i, clf in enumerate(results_table):
    print "** Classifier {} - {} **".format(i + 1, clf)
    print ""
    print "| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |"
    print "| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |"
    for results in results_table[clf]:
        print "| {0:d}               | {1:.4f}                      |{2:.4f}                       |{3:.4f}               |{4:.4f}               |".format(*results)
    print ""

In [10]:
from sklearn.metrics import f1_score
print "F1 score for predicting all \"yes\" on test set: {:.4f}".format(
    f1_score(y_test, ['yes']*len(y_test), pos_label='yes', average='binary'))

F1 score for predicting all "yes" on test set: 0.8050


** Classifier 1 - BaggingClassifier **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0635                      |0.0118                       |0.8667               |0.7919               |
| 200               | 0.0795                      |0.0107                       |0.8562               |0.7947               |
| 300               | 0.1167                      |0.0304                       |0.8825               |0.8158               |

** Classifier 2 - SVC **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0075                      |0.0013                       |0.8354               |0.8025               |
| 200               | 0.0074                      |0.0029                       |0.8431               |0.8105               |
| 300               | 0.0090                      |0.0022                       |0.8664               |0.8052               |

** Classifier 3 - KNeighborsClassifier **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0008                      |0.0017                       |0.8252               |0.7586               |
| 200               | 0.0017                      |0.0051                       |0.8097               |0.7857               |
| 300               | 0.0057                      |0.0042                       |0.8539               |0.8138               |


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Since the classes are inbalanced with more number of passing students than failing, I decided to use a benchmark model of predicting yes to all the rows. The benchmark on the test set by just predicting yes comes out to be 0.8050.

The bagging classifier acheives the highest f1 scores on the trainig as well as test sets when trained on the entire training set. the algorithm however acheives slightly lower f1 scores for the smaller training sets. it 2 order of magnitudes higher training and testing time than the individual SVM, KNN classifiers. This makes sense because the bagging ensemble method trains multiple weak classifiers and then aggregates the results to make the final prediction.

The SVM classifier performs well consistently on all the training sets, acheiving higher than benchmark f1 scores. it appears that the SVM slightly overfitted on the 300 sample training set since the testing score decresed but the training score increased. The training time and testing time were lower than 0.01 seconds and was atleast twice as fast as the bagging ensemble of SVMs

The KNN classifer had the fastest training time but was 2 times slower than the SVM during testing runs, it acheives the lowest f1 scores during training time but the final test f1 score after training on  the 300 sample training set is comparable to the ensemble of SVMs trained.

All the algorithms performed quite close to the benchmark of predicting all yes. Since the training time of the algorithms arent very high and all three train in less than 1/2 a second I decided that the ensemble of SVMs was the best model to choose since it acheives the highest f1 score on the final set. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **


I chose to use the ensemble of SVMs as the final chosen model.

![title](bagging.png)
1. The bagging algorithm takes the past data of the students like age, gender, family size etc. and splits the dataset  into random subsets such as a subset of only age and gender, family and travel time etc.

2. It then assigns an SVM to each subset which was created from the split, The SVM takes the data which was assigned to it and tries to create a boundary between students who graduated and students who did not. The boundary drawn by the SVM tries to maximize the margin between the two groups so as to be robust for future classifications.
    ![title](optimal-hyperplane.png)
3. Training multiple SVMs on each subset is analogous to having a group of experts who make predictions using only the subset of features that were assigned to them. 

4. The predictions of the multiple SVMs are taken as a vote for the final prediction made by the algorithm. 


This voting from multiple algorithms is a very useful strength of the algorithm as it prevents the algorithm from overfitting ( memorizing ) the past data and tends to give better performance than training an individual algorithm.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [35]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = {
    "max_samples": [0.5,1],
    "max_features": [0.5, 1],
    "bootstrap": [True, False],
}


# TODO: Initialize the classifier
clf = ensemble.BaggingClassifier(svm.SVC(), random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, 
                        param_grid=parameters,
                        scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
train_pred , train_time = predict_labels(clf, X_train, y_train)
test_pred, test_time = predict_labels(clf, X_test, y_test)
print "Tuned model has a training F1 score of {:.4f}.".format(train_pred)
print "Tuned model has a testing F1 score of {:.4f}.".format(test_pred)

Made predictions in 0.0262 seconds.
Made predictions in 0.0101 seconds.
Tuned model has a training F1 score of 0.8372.
Tuned model has a testing F1 score of 0.8182.


In [36]:
clf.get_params()


{'base_estimator': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 'base_estimator__C': 1.0,
 'base_estimator__cache_size': 200,
 'base_estimator__class_weight': None,
 'base_estimator__coef0': 0.0,
 'base_estimator__decision_function_shape': None,
 'base_estimator__degree': 3,
 'base_estimator__gamma': 'auto',
 'base_estimator__kernel': 'rbf',
 'base_estimator__max_iter': -1,
 'base_estimator__probability': False,
 'base_estimator__random_state': None,
 'base_estimator__shrinking': True,
 'base_estimator__tol': 0.001,
 'base_estimator__verbose': False,
 'bootstrap': False,
 'bootstrap_features': False,
 'max_features': 0.5,
 'max_samples': 0.5,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [37]:
clf_C.get_params()

{'base_estimator': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 'base_estimator__C': 1.0,
 'base_estimator__cache_size': 200,
 'base_estimator__class_weight': None,
 'base_estimator__coef0': 0.0,
 'base_estimator__decision_function_shape': None,
 'base_estimator__degree': 3,
 'base_estimator__gamma': 'auto',
 'base_estimator__kernel': 'rbf',
 'base_estimator__max_iter': -1,
 'base_estimator__probability': False,
 'base_estimator__random_state': None,
 'base_estimator__shrinking': True,
 'base_estimator__tol': 0.001,
 'base_estimator__verbose': False,
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final f1 score of the ensemble of SVMs is 0.8182 which is higher than the f1 scores achieved by the untuned model.  the training f1 score is lower than the untuned model. this suggests that the tuned model has better generalized on the data than the untuned model. it also beats the benchmark score of 0.8050.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.